# 02 - Vocabulary Building and Data Loading

## Learning Objectives
- Build vocabulary from cleaned text data
- Create efficient PyTorch data loaders
- Implement text-to-sequence conversion
- Handle variable-length sequences
- Understand PyTorch Dataset and DataLoader concepts

## Phase 1: PyTorch Fundamentals 🧠
*Build everything from scratch to understand the foundations*

## Phase 2: Transformers Enhancement 🚀
*Enhance with modern NLP tools after mastering fundamentals*

---

## Overview

In this notebook, you'll build the data pipeline that converts your cleaned text into numerical sequences that PyTorch models can process. This involves:

1. **Vocabulary Building**: Creating word-to-index mappings
2. **Text-to-Sequence Conversion**: Converting text to numerical sequences
3. **PyTorch Dataset**: Creating custom dataset classes
4. **Data Loading**: Implementing efficient data loaders with batching
5. **Pipeline Validation**: Testing and optimizing your data pipeline

## Prerequisites

Make sure you have completed:
- ✅ `00_exploration.ipynb` - Data exploration and EDA
- ✅ `01_preprocessing.ipynb` - Text cleaning and preprocessing

You'll be working with the cleaned datasets from `data/interim/` folder.

---

## TODO 1: Build Vocabulary

**Goal**: Create word-to-index mapping from cleaned text

**Steps**:
1. Load cleaned text data from `data/interim/`
2. Tokenize text into words
3. Build vocabulary with special tokens:
   - `<PAD>` for padding (index 0)
   - `<UNK>` for unknown words (index 1)
   - `<START>` and `<END>` tokens (optional)
4. Calculate vocabulary statistics

**Hint**: Use `Counter` to count word frequencies, set minimum frequency threshold

**Expected Output**: Word-to-index mapping and vocabulary size


In [35]:
# TODO 1: Build vocabulary
# Your implementation here
from collections import Counter
import pandas as pd

cleaned_text = pd.read_csv("../data/interim/train_cleaned.csv")
extracted_text = cleaned_text["text_clean"]
print(extracted_text.head())
print(extracted_text.dtypes)
print(extracted_text.isna().sum())
extracted_text = cleaned_text["text_clean"].dropna()
print(extracted_text.isna().sum())


extracted_words = extracted_text.apply(lambda x: x.split())
print(extracted_words.head())
count_words = Counter(word for words in extracted_words for word in words)
print(count_words.most_common(10))
vocab = sorted(count_words.keys())
vocab_size = len(vocab)

print(f"Vocabulary size: {vocab_size}")
print(f"First 10 words: {vocab[:25]}")
print(f"Last 10 words: {vocab[-10:]}")


0            deeds reason earthquake may allah forgive
1                   forest fire near ronge sask canada
2    residents asked shelter place notified officer...
3    thirteenzero people receive wildfires evacuati...
4    got sent photo ruby alaska smoke wildfires pou...
Name: text_clean, dtype: object
object
0
0
0     [deeds, reason, earthquake, may, allah, forgive]
1            [forest, fire, near, ronge, sask, canada]
2    [residents, asked, shelter, place, notified, o...
3    [thirteenzero, people, receive, wildfires, eva...
4    [got, sent, photo, ruby, alaska, smoke, wildfi...
Name: text_clean, dtype: object
[('hundred', 540), ('one', 439), ('two', 435), ('thousand', 417), ('like', 345), ('amp', 298), ('fire', 250), ('get', 229), ('new', 224), ('via', 220)]
Vocabulary size: 14644
First 10 words: ['UNK', 'aaaa', 'aaaaaaallll', 'aaaaaand', 'aaarrrgghhh', 'aampb', 'aampw', 'aan', 'aannnnd', 'aar', 'aashiqui', 'aba', 'abandon', 'abandoned', 'abandoning', 'abbandoned', 'abbott', 'a

## TODO 2: Text to Sequence Conversion

**Goal**: Convert text to numerical sequences

**Steps**:
1. Implement function to convert text to sequence of indices
2. Handle unknown words with `<UNK>` token
3. Add padding to sequences for batch processing
4. Test conversion on sample texts

**Hint**: Use vocabulary mapping and consider sequence length limits

**Expected Output**: Numerical sequences ready for model input


In [40]:
# TODO 2: Text to sequence conversion
# Your implementation here


# Make sure <PAD> is at index 0 and <UNK> is at index 1 in the vocab
vocab = ['<PAD>', '<UNK>'] + [word for word in vocab if word not in ('<PAD>', '<UNK>')]
vocab_dict = {word: idx for idx, word in enumerate(vocab)}


def text_to_sequence(text, vocab_dict, max_length=50, pad_idx=0, unk_idx=1):
    """
    Convert text to sequence of indices using vocabulary mapping.
    
    Args:
        text (str): Input text to convert
        vocab_dict (dict): Vocabulary mapping (expects <PAD> at 0, <UNK> at 1)
        max_length (int): Maximum sequence length
        pad_idx (int): Index for padding token (should be 0)
        unk_idx (int): Index for unknown token (should be 1)
        
    Returns:    
        list: Sequence of indices
    """
    # Convert text to lowercase and split into words
    words = text.lower().split()
    
    # Convert words to indices using vocabulary mapping
    sequence = [vocab_dict.get(word, unk_idx) for word in words]
    
    # Pad or truncate sequence to max_length
    if len(sequence) < max_length:
        sequence.extend([pad_idx] * (max_length - len(sequence)))
    else:
        sequence = sequence[:max_length]
    
    return sequence

# Test the function
test_text = "This is a test tweet with some unknown words and a disaster."
print(text_to_sequence(test_text, vocab_dict))




[1, 1, 1, 12702, 13315, 1, 1, 13528, 14280, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## TODO 3: Custom PyTorch Dataset

**Goal**: Create PyTorch Dataset class for disaster tweets

**Steps**:
1. Inherit from `torch.utils.data.Dataset`
2. Implement `__len__` and `__getitem__` methods
3. Handle text-to-sequence conversion
4. Return tensors for text and labels

**Hint**: Use `torch.tensor()` for tensor creation

**Expected Output**: Custom dataset class ready for DataLoader


In [ ]:
# TODO 3: Custom PyTorch dataset
# Your implementation here
import torch
from torch.utils.data import Dataset

class DisasterTweetsDataset(Dataset):
    def __init__(self, texts, labels, vocab_dict, max_length=50):
        self.texts = texts
        self.labels = labels


## TODO 4: Data Loading and Batching

**Goal**: Create efficient data loaders with proper batching

**Steps**:
1. Create train/validation/test splits
2. Implement custom collate function for variable-length sequences
3. Create DataLoaders with appropriate batch sizes
4. Test data loading pipeline

**Hint**: Use `torch.nn.utils.rnn.pad_sequence` for padding

**Expected Output**: Efficient data loading pipeline ready for training


In [4]:
# TODO 4: Data loading and batching
# Your implementation here


## TODO 5: Vocabulary Analysis and Optimization

**Goal**: Analyze vocabulary characteristics and optimize for modeling

**Steps**:
1. Calculate vocabulary statistics (size, coverage, frequency distribution)
2. Analyze sequence length distribution
3. Determine optimal sequence length for padding/truncation
4. Visualize vocabulary and sequence statistics
5. Save vocabulary for later use

**Hint**: Consider vocabulary size impact on model performance and memory usage

**Expected Output**: Optimized vocabulary ready for model training


In [5]:
# TODO 5: Vocabulary analysis and optimization
# Your implementation here


## TODO 6: Data Pipeline Testing and Validation

**Goal**: Validate the complete data pipeline before moving to modeling

**Steps**:
1. Test data loading with sample batches
2. Verify tensor shapes and data types
3. Check for any data inconsistencies
4. Measure data loading performance
5. Document pipeline characteristics

**Hint**: Use `torch.utils.data.DataLoader` with `num_workers` for efficiency

**Expected Output**: Validated data pipeline ready for model training

---

## Phase 2: Transformers Enhancement

*After completing Phase 1, consider these enhancements:*

- Use HuggingFace tokenizers for consistent tokenization
- Leverage pre-trained vocabularies (BERT, RoBERTa)
- Compare custom vocabulary vs. pre-trained tokenizers
- Analyze vocabulary size impact on model performance


In [6]:
# TODO 6: Data pipeline testing and validation
# Your implementation here
